# Q1. Getting the embeddings model

## What's the first value of the resulting vector for model multi-qa-distilbert-cos-v1?


- -0.24
- -0.04
- 0.07
- 0.27

In [96]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [97]:
user_question = "I just discovered the course. Can I still join it?"

In [98]:
user_question_emb = embedding_model.encode(user_question)

In [99]:
user_question_emb[0]

0.07822261

## Answer:
0.07

## Prepare documents  
Load the documents with ids that we prepared in the module  
We will use only a subset of the questions - the questions for "machine-learning-zoomcamp"  
After filtering, you should have only 375 documents

In [100]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [101]:
def embedding_question_text(doc):
    doc['qa_text'] = embedding_model.encode(f'{doc["question"]} {doc["text"]}')
    return doc

In [102]:
documents_ml_course = [embedding_question_text(i) for i in documents if i['course'] == 'machine-learning-zoomcamp' ]

In [103]:
len(documents_ml_course)

375

# Q2. Creating the embeddings

In [104]:
import numpy as np

In [105]:
embeddings = [embdoc['qa_text'] for embdoc in documents_ml_course]

In [106]:
X = np.array(embeddings)

In [107]:
X.shape

(375, 768)

## Answer
(375, 768)

# Q3. Search

In [108]:
v = user_question_emb
scores = X.dot(v)

In [109]:
np.max(scores)

0.65065753

## Answer
0.65

# Q4. Hit-rate for our search engine

In [ ]:
from tqdm.auto import tqdm

In [110]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)

In [111]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [112]:
ground_truth

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Is there an FAQ for this Machine Learning course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Does this course have a GitHub repository for the sign-up link?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'How can I structure my questions and answers for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Are the course videos live or pre-recorded?',
  'course': 'machine-learning-zoomcamp',
  'document': '39fda9f0'},
 {'question': 'When can I start watching the course videos?',
  'course': 'machine-learning-zoomcamp',
  'document': '39fda9f0'},
 {'question': 'Are the live office hours sessions recorded?',
  'cours

In [113]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [114]:
search_engine = VectorSearchEngine(documents=documents_ml_course, embeddings=X)

In [115]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [116]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [117]:
def search(document):
    question = document['question']
    v_query = embedding_model.encode(question)
    return search_engine.search(v_query, num_results=5)

In [118]:
evaluation_result = evaluate(ground_truth, search)

  0%|          | 0/1830 [00:00<?, ?it/s]

100%|██████████| 1830/1830 [00:32<00:00, 55.68it/s]


In [119]:
evaluation_result

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

## Answer
0.93

# Q5. Indexing with Elasticsearch

In [ ]:
! docker run -it -d \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [120]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

In [125]:
es_client.indices.delete(index=index_name)

ObjectApiResponse({'acknowledged': True})

In [126]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "qa_text": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "ml-course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ml-course-questions'})

In [127]:
for doc in tqdm(documents_ml_course):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:02<00:00, 173.61it/s]


In [128]:
search_query = {
        "size": 500,
        "query": {
            "bool": {
                "must": {
                    "match_all": {}
                }
            }
        }
    }

In [129]:
response = es_client.search(index=index_name, body=search_query)
len(response['hits']['hits'])

375

In [130]:
response['hits']['hits'][0]['_source']

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872',
 'qa_text': [0.08805911988019943,
  0.015593653544783592,
  0.07925572246313095,
  0.025275781750679016,
  0.07557645440101624,
  -0.03905962407588959,
  -0.04138132557272911,
  0.025291718542575836,
  0.024324148893356323,
  0.003625929355621338,
  -0.007282918784767389,
  -0.03287506848573685,
  0.06129564717411995,
  -0.057110078632831573,
  0.011677442118525505,
  -0.01794414408504963,
  0.04492054507136345,
  -0.05416063964366913,
  -

In [131]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [132]:
response = elastic_search_knn('qa_text', user_question_emb, 'machine-learning-zoomcamp')


In [133]:
response[0]

{'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'section': 'General course-related questions',
 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'id': 'ee58a693'}

## Answer
ee58a693

# Q6. Hit-rate for Elasticsearch